# WEB SCRAPING
  
Es una técnica para extraer y almacenar datos de una o varias páginas web con el fin de analizarlos o manipularlos en otros medios, para la cual se utilizan bots para extraer los datos y contenidos de las webs. 

### Importación de librerías.

* Requests realiza la petición al servidor.

* BeautifulSoup analizar documentos HTML.

* Pandas podemos representar datos tabulares con columnas con etiquetas y filas y series temporales.

In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

Se realiza un request de la url_inicial de la revista, url_root nos ayuda a genera un url completa a partir de la url raíz para esto se utiliza la librería urljoin.

La revista que se está scrapendo es la siguiente https://geotermia.org.mx/

In [ ]:
url_inicial='https://geotermia.org.mx/revista/page/4/#download'
url_root= 'https://geotermia.org.mx/revista/page/4/#download'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

### Obtención de los primeros volúmenes.

Se realiza la búsqueda para obtener los urls de cada uno de los volúmenes o archivos de la revista.

In [ ]:
box = soup.find('div', class_='col-xs-12 col-sm-9')
volu=box.findAll('div', class_='col-xs-12 col-sm-4')
vol = [x.find('a').get('href')for x in volu]
#vol=[urljoin(url_root,i) for i in vol]
vol

En esta función variable sopa se utiliza para tener las paginaciones de la revista y la variable url es guardan cada uno de url que ya se obtuvieron de *vol*.

In [ ]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='col-xs-12 col-sm-9')
    volu=box.findAll('div', class_='col-xs-12 col-sm-4')
    vol = [x.find('a').get('href')for x in volu]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

Ahora se va acumulando cada uno de los links e ir iterando en cada una de las páginas, para traer cada uno de los links que se van a ir scrapeando.

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Obtiene el número de los artículos que se encontraron en los primeros volúmenes de la revista.

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

Toma uno a uno de los links donde se encuentra la información de cada artículo para Scrapearlos.

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

### Scrapeo de las urls de los ariculos.

Mediante esta función revisa cada uno de los links de los artículos en los cuales se encuentra la información
del artículo, por lo cual se aplican cada uno de los métodos, que nos van a obtener lo que estamos requiriendo de dicho artículo.

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Revista Mexicana de Enfermería Cardiológica'
    a='III. Medicina y Ciencias de la Salud'
    te='Nursing'
    ta='https://geotermia.org.mx/revista/'
    ta1='https://geotermia.org.mx/revista/page/3/#download'
    lin= vol
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('div', class_='col-xs-12 col-sm-4').find('a').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', class_='panel-body').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('div', class_='panel-boody').get_text(strip=True)
        content_book['Abstract']=de#.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    try:
        urlA=None
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
        #obtener link PDF
    try:
        ancla_link=url
        content_book['Link PDF']=ancla_link
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

El list_scraper hace un scrapeo de la cada uno de los artículos, ya que en datos_book se acumuló la información obtenida de lo métodos anteriores.

In [ ]:
list_scraper=list_scraper[0:1]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

### Se obtiene la información requerida de los artículos.

La variable **datos_book** es un listado en el que se recolecto la información requerida por lo cual
se convierte en un **DataFrame**.

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

La información solicitada se pasa a un **CSV**.

In [ ]:
df_catalogo.to_csv('Revista100.2.csv', index=False)

Final del Código